In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
linkTrain = 'https://drive.google.com/file/d/1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI/view?usp=sharing'
linkIdTrain = '1uV8qsVnl1XLxLlfw2tnGF3rbIU3tntHI'

linkTest = 'https://drive.google.com/file/d/1QIJWtK99SblVQK88AHqHwAVBZJcT14yW/view?usp=sharing'
LinkIdTest = '1QIJWtK99SblVQK88AHqHwAVBZJcT14yW'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
downloaded = drive.CreateFile({'id':linkIdTrain}) 
downloaded.GetContentFile('Bot_IOT_Train.csv')  

downloaded = drive.CreateFile({'id':LinkIdTest}) 
downloaded.GetContentFile('Bot_IOT_Test.csv')  

#loading data
df_nb_Train = pd.read_csv('Bot_IOT_Train.csv')
df_nb_Test = pd.read_csv('Bot_IOT_Test.csv')

In [ ]:
#Data preprocessing
#checking for NULL Values for training
null_columns = df_nb_Train.columns[df_nb_Train.isnull().any()]
df_nb_Train[null_columns].isnull().sum()

#for testing
null_columns = df_nb_Test.columns[df_nb_Test.isnull().any()]
df_nb_Test[null_columns].isnull().sum()

Series([], dtype: float64)

In [ ]:
#ignoring non important attributes from dataset
#training
df_nb_Train=df_nb_Train.drop(['pkSeqID', 'saddr', 'sport', 'daddr', 'dport', 'proto'], axis = 1)

#for testing
df_nb_Test=df_nb_Test.drop(['pkSeqID', 'saddr', 'sport', 'daddr', 'dport', 'proto'], axis = 1)

In [ ]:
# import machine learning algorithms
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#dependent and independent variables
X_Train = df_nb_Train.iloc[:, 0:10]
Y_Train = df_nb_Train.loc[:, 'attack']


X_Test = df_nb_Test.iloc[: , 0:10]
Y_Test = df_nb_Test.loc[:,'attack']

from imblearn.over_sampling import SMOTE                        # Balance imbalance data
sm = SMOTE(random_state=2)
X_Train, Y_Train = sm.fit_sample(X_Train, Y_Train.ravel())
print(sum(Y_Train==0))
print(sum(Y_Train==1))

print(X_Train.shape)
print(Y_Train.shape)
print(X_Test.shape)
print(Y_Test.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


2934447
2934447
(5868894, 10)
(5868894,)
(733705, 10)
(733705,)


In [ ]:
model_nb = GaussianNB()

In [ ]:
model_nb.fit(X_Train, Y_Train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
#predicting
y_pred = model_nb.predict(X_Test)

In [ ]:
#model evaluation
accuracy = accuracy_score(Y_Test,y_pred)*100
accuracy

99.30203555925065

In [ ]:
#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_Test, y_pred))

Confusion Matrix:
[[   107      0]
 [  5121 728477]]


In [ ]:
print("Classification Report")
print(classification_report(Y_Test, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.02      1.00      0.04       107
           1       1.00      0.99      1.00    733598

    accuracy                           0.99    733705
   macro avg       0.51      1.00      0.52    733705
weighted avg       1.00      0.99      1.00    733705



In [ ]:
#encoding 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
df_nb_Train['category']=le.fit_transform(df_nb_Train['category'])
df_nb_Train['subcategory']=le1.fit_transform(df_nb_Train['subcategory'])

df_nb_Test['category']=le.transform(df_nb_Test['category'])
df_nb_Test['subcategory']=le1.transform(df_nb_Test['subcategory'])

In [ ]:
#dependent and independent variables
X_cat_train = df_nb_Train.iloc[:, 0:11]
Y_cat_train = df_nb_Train.loc[:, 'category']

X_cat_test =df_nb_Test.iloc[:, 0:10]
X_cat_test['attack']= y_pred
Y_cat_test=df_nb_Test.loc[:, 'category']

print(X_cat_train)
print(Y_cat_train)

            seq    stddev  N_IN_Conn_P_SrcIP  ...     srate       max  attack
0        251984  1.900363                100  ...  0.494549  4.031619       1
1        256724  0.078003                 38  ...  0.256493  4.012924       1
2         62921  0.268666                100  ...  0.294880  3.609205       1
3         99168  1.823185                 63  ...  0.461435  4.942302       1
4        105063  0.822418                100  ...  1.002999  4.994452       1
...         ...       ...                ...  ...       ...       ...     ...
2934812  253370  0.016992                100  ...  0.243473  4.124047       1
2934813  231693  1.922317                100  ...  0.490800  4.090534       1
2934814  158616  2.112228                100  ...  0.207444  4.332615       1
2934815  179855  0.000000                100  ...  0.162130  0.000000       1
2934816   95429  0.053820                100  ...  0.116388  0.172408       1

[2934817 rows x 11 columns]
0          0
1          0
2        

In [ ]:
#classifier
model_nb_cat = GaussianNB()
model_nb_cat.fit(X_cat_train, Y_cat_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
#calcuating accuracy of the model on test set
pred_cat = model_nb_cat.predict(X_cat_test)

#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_cat_test, pred_cat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_cat_test, pred_cat))

#classification report
print("Classification Report")
print(classification_report(Y_cat_test, pred_cat))

Accuracy Score: 
70.86635636938551
Confusion Matrix:
[[384470    839      0      0      0]
 [196635 133472      1      4      0]
 [     0     94      1      4      8]
 [ 10913   5237      5   2007      1]
 [     0     14      0      0      0]]
Classification Report
              precision    recall  f1-score   support

           0       0.65      1.00      0.79    385309
           1       0.96      0.40      0.57    330112
           2       0.14      0.01      0.02       107
           3       1.00      0.11      0.20     18163
           4       0.00      0.00      0.00        14

    accuracy                           0.71    733705
   macro avg       0.55      0.30      0.31    733705
weighted avg       0.80      0.71      0.67    733705



In [ ]:
####################Predicting subcategory of attack
#dependent and independent variables
X_subcat_train = df_nb_Train.iloc[:, 0:12]
Y_subcat_train = df_nb_Train.loc[:, 'subcategory']

X_subcat_test =df_nb_Test.iloc[:, 0:10]

X_subcat_test['attack']= y_pred
X_subcat_test['category']= pred_cat
Y_subcat_test=df_nb_Test.loc[:, 'subcategory']

In [ ]:
#classifying
model_nb_subcat = GaussianNB()
model_nb_subcat.fit(X_subcat_train, Y_subcat_train) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
#calcuating accuracy of the model on test set
pred_subcat = model_nb_subcat.predict(X_subcat_test)

#calculate accuracy
print("Accuracy Score: ")
print (accuracy_score(Y_subcat_test, pred_subcat)*100)

#confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(Y_subcat_test, pred_subcat))

Accuracy Score: 
81.37085068249502
Confusion Matrix:
[[     0      0      0      0      0      0      0      0]
 [     0    242      0      0      0      0    239     23]
 [     1     13      0      0      0      0      0      0]
 [    10     11      0      1      0     38     36     11]
 [     0     32      0      2     48     93   3434     12]
 [     0    626      0      3    706   1241  10414   1552]
 [     2    354      0      0      0     67 198999 118915]
 [     4     67      0      0      0      3     15 396491]]


In [ ]:
#df['attack', 'category', 'subcategory']
a=df_nb_Test['attack']
b=df_nb_Test['category']
c=df_nb_Test['subcategory']
count=0
for i in range(0,len(df_nb_Test)):
  if(y_pred[i]==a[i]) and (pred_cat[i]==b[i]): #and (pred_subcat[i] == c[i]) :
    count=count+1
count
count/len(df_nb_Test)*100

70.45052166742765